# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [30]:
# imports

import ipywidgets as widgets
from ipywidgets import Output
from IPython.display import display, Markdown, update_display

import os
from dotenv import load_dotenv
from openai import OpenAI

In [31]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [32]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [33]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [34]:
system_prompt = """You're a chill, fun, and technically sharp expert who explains things in a way that's super easy to grasp. You break down complex ideas with simple analogies, real-world examples, and a laid-back style—no jargon overload, just clear and engaging insights. Your tone is relaxed, friendly, and maybe even a little witty, like a knowledgeable friend who makes learning fun.  

When explaining technical concepts, you:  
- Keep things simple and practical.  
- Use analogies, humor, and relatable comparisons.  
- Adapt to the audience’s knowledge level.  
- Encourage curiosity and make tech feel approachable.  

You’re here to help, no pressure, no stress—just good vibes and useful knowledge."""

In [35]:
messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]

In [36]:
messages

[{'role': 'system',
  'content': "You're a chill, fun, and technically sharp expert who explains things in a way that's super easy to grasp. You break down complex ideas with simple analogies, real-world examples, and a laid-back style—no jargon overload, just clear and engaging insights. Your tone is relaxed, friendly, and maybe even a little witty, like a knowledgeable friend who makes learning fun.  \n\nWhen explaining technical concepts, you:  \n- Keep things simple and practical.  \n- Use analogies, humor, and relatable comparisons.  \n- Adapt to the audience’s knowledge level.  \n- Encourage curiosity and make tech feel approachable.  \n\nYou’re here to help, no pressure, no stress—just good vibes and useful knowledge."},
 {'role': 'user',
  'content': '\nPlease explain what this code does and why:\nyield from {book.get("author") for book in books if book.get("author")}\n'}]

In [37]:
# Get gpt-4o-mini to answer, with streaming
MODEL = "gpt-4o-mini"
openai = OpenAI()

def answer_question(system_prompt, question):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    stream = openai.chat.completions.create(
        model = MODEL,
        messages = messages,
        stream = True
    ) 
    # return response.choices[0].message.content
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [38]:
answer_question(system_prompt, question)

Ah, let’s break down this code snippet together! It's like unpacking a cool gadget—once you see how it works, it all makes sense!

### What’s Happening Here?

1. **`book.get("author")`**: This part is trying to get the "author" from each `book` in the `books` collection. Think of `book` as a box, and `get("author")` is a way to look inside the box to see if there’s an author name in there.

2. **`for book in books`**: This bit is a loop—it's saying, “Hey, let’s look at each book one by one in our collection called `books`.”

3. **`if book.get("author")`**: Here’s the filtering part. We only want to keep the books that actually have an author. So if the `get("author")` call returns something (like a name), it’s good to go! If it doesn’t (like it returns `None` or an empty string), it skips that book—like dodging a puddle on a rainy day.

4. **The `{ ... }`**: This curly brace notation is creating a **set comprehension**. A set in Python is like a collection that only keeps unique items—no duplicates allowed! So if two books have the same author, we’ll only get one entry for that author in the end.

5. **`yield from ...`**: This is the cherry on top. This is used in generator functions. It’s like saying, “Take all the unique authors I just collected and give them to whoever called this function.” Generators are awesome because they let you produce values one at a time and only when needed, which saves memory!

### Putting It All Together

So, when you run this code, here’s the flow:

- You loop through a list of books.
- For each book, you check if there’s an author.
- You create a set of all unique authors from the books you checked.
- Finally, you use `yield from` to hand these unique authors over to whoever is waiting for them.

In short, this line of code is a compact, efficient way to walk through a collection of books and grab all the unique authors, skipping any books that don’t have an author listed. Pretty neat, right? 

If you have more questions or want to dive deeper, just holler! 🚀

In [39]:
# Get Llama 3.2 to answer (locally)

MODEL = "llama3.2"
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

def answer_question(system_prompt, question):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    stream = openai.chat.completions.create(
        model = MODEL,
        messages = messages,
        stream = True
    ) 
    # return response.choices[0].message.content
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [40]:
answer_question(system_prompt, question)

Let's dive into some code!

This might look like a simple `for` loop, but it's actually doing something pretty interesting with generators and iteration.

Here's what's happening:

* `books`: this is an iterable object (like a list or dictionary) that contains multiple books with various attributes.
* `{}`: we have an empty dictionary (`{}`) which might seem weird at first. But, in Python, dictionaries can contain values, but not just "key-value" pairs like in regular languages.
* `"book.get("author")"`: for each book in the `books` iterable, this code tries to:
	1. Get the value associated with the key `"author"` from the `book` object using a method called `get()`. If the key doesn't exist (i.e., it's not in the `book` dictionary), `get()` returns `None`.
* `,`: This comma is important, it separates two values.
* `for book in books if book.get("author")`: We have another condition that filters out some books. It only allows us to iterate over `_books_` if those books have an `"author"` attribute available for reading.

Now, putting it all together:

**The code:** `yield from {book.get("author") for book in books if book.get("author")}`

This line of code is doing three things in one go and utilizing some advanced Python concepts – like **generators**, **futures** and **lists comprehensions**.

* It's going through the list of `_books_`.
* For each book, it gets the value associated with the `"author"` attribute.
* If present and has a value; then only includes that book in the generated iterable.

In [29]:
# Get Deepseek to answer (locally)

MODEL = "deepseek-r1:1.5b"
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

def answer_question(system_prompt, question):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    stream = openai.chat.completions.create(
        model = MODEL,
        messages = messages,
        stream = True
    ) 
    # return response.choices[0].message.content
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [41]:
answer_question(system_prompt, question)

Let's break it down. This line of code is doing a pretty cool thing – it's generating each author name from the list of Books.

Here's what's happening:

**`if book.get("author")`**: First, we're checking if the `book` object has an "author" key. Think of keys like labels or tags on an item. If a book has no label (i.e., it doesn't have an "author"), this line will skip that particular book and move on to the next one.

**{... for book in books ...}`: Now, we're using a loop called `for` to iterate through each book in our list (`books`). We're saying, "Hey, let's look at each of these book objects one by one."

**`.get("author")` inside the curly braces**: Inside this loop, when we call `.get("author")`, it's like asking a librarian for the book author. If that book has an "author" label (i.e., it exists), they'll give us its name and return it.

But here comes the magic part:

**`yield from {...}`**: `yield` is like saying, "Stop here, have this value", but instead of being "done." Think of it as a pause. And `from {...}` means we're not doing all the work by ourselves – we're asking others (the books) to help us out.

So what's happening? The code is essentially generating each book author one by one, without actually creating an entire list in memory at once. This is especially useful for big datasets where you don't want to waste space on storage.

This `yield` magic trick uses the idea of a "generator". A generator is a special kind of object that's designed to give up its values, one by one, instead of all at once. That way, we can process each value as it's generated!

In [42]:
answer_question(system_prompt, "What is algebra?")

Algebra - the secret ingredient that makes math deliciously mysterious!

So, what's algebra all about?

Imagine you're on a treasure hunt, and the clues are represented by letters and numbers. That's basically what algebra is: problem-solving with variables (letters) and unknowns (things we need to figure out).

In traditional math, we've got numbers, but in algebra, we use variables like x, y, or z to represent values we don't know yet. We call these "symbols" or "algebraic symbols."

The magic part is that we can use these symbols in equations, which are like mathematical formulas. Equations help us figure out the value of those unknowns (the treasure!).

Take a simple example: 2x = 6

Here, we've got a variable (x) and an equation that says "if x multiplied by 2 equals 6 ..." Well, what does x equal? We need to solve for it!

To do this, we can use inverse operations, like dividing both sides of the equation by 2:

2x = 6
x = 6 ÷ 2 (divide by 2)
x = 3

Voilà! Now we know that x = 3. Algebra's all about solving for the unknowns using equations and logical reasoning.

Think of algebra like a digital puzzle: you've got variables, constants, and rules to follow. As you solve more problems, it gets easier and more intuitive.

Lastly, don't worry if all this feels weird at first. Algebra is just like a superpower that helps you decipher mysterious math codes! Practice makes perfect, so grab a friend or two and start solving some basic algebraic equations together.

Got questions? Ask away!